In [1]:
import pyspark

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
# Jugadores = (id_jugador, nombre_jugador, fecha_nacimiento, pais)
# Estadisticas = (id_partido, id_jugador, tiempo_juego, distancia_recorrida, goles)
# Partidos = (id_partido, año, semestre, nro_fecha, equipo_local, equipo_visitante)

# Conseguir un rdd con los jugadores nacidos en Argentina que hayan aumentado en mas de 50% el tiempo que jugaron de un
# semestre a otro.

In [3]:
jugadores = [
    (1, 'Maradona, Diego', '30/10/1960', 'Argentina'),
    (2, 'Messi, Lionel', '24/06/1986', 'Argentina'),
    (3, 'Do Nascimiento, Edson', '23/10/1940', 'Brazil')
]

estadisticas = [
    (1, 1, 20, 1200, 2),
    (1, 3, 30, 600, 1),
    (2, 1, 42, 1600, 1),
    (2, 3, 90, 2200, 0),
    (3, 1, 65, 3100, 2),
    (4, 2, 40, 1200, 2),
    (5, 2, 85, 1100, 3)
]

partidos = [
    (1, 1978, 1, 1, 'ARG', 'BRA'),
    (2, 1978, 2, 4, 'BRA', 'ARG'),
    (3, 1984, 1, 2, 'ARG', 'ENG'),
    (4, 2009, 1, 1, 'Barcelona', 'Real Madrid'),
    (5, 2015, 2, 5, 'ARG', 'HOL')
]

jugadores_rdd = sc.parallelize(jugadores)
estadisticas_rdd = sc.parallelize(estadisticas)
partidos_rdd = sc.parallelize(partidos)

partidos_rdd.take(5)

[(1, 1978, 1, 1, 'ARG', 'BRA'),
 (2, 1978, 2, 4, 'BRA', 'ARG'),
 (3, 1984, 1, 2, 'ARG', 'ENG'),
 (4, 2009, 1, 1, 'Barcelona', 'Real Madrid'),
 (5, 2015, 2, 5, 'ARG', 'HOL')]

In [6]:
def filter_func(dates_with_avgs):
    # [(year, quarter), avg]
    for i in range (1, len(dates_with_avgs)):
        if dates_with_avgs[i][0][0] == dates_with_avgs[i-1][0][0]+1 and dates_with_avgs[i][0][1] == dates_with_avgs[i-1][0][1]-1:
            return (dates_with_avgs[i][1]/dates_with_avgs[i-1][1] >= 1.5)
        elif dates_with_avgs[i][0][0] == dates_with_avgs[i-1][0][0] and dates_with_avgs[i][0][1] == dates_with_avgs[i-1][0][1]+1:
            return (dates_with_avgs[i][1]/dates_with_avgs[i-1][1] >= 1.5)
    return False
        
jugadores_argentinos = jugadores_rdd.filter(lambda x: x[3] == 'Argentina')\
                                       .map(lambda x: (x[0], x[1])) #id, nombre

partidos = partidos_rdd.map(lambda x: (x[0], (x[1], x[2])))

estadisticas_argentinos = estadisticas_rdd.map(lambda x: (x[1], (x[0], x[2])))\
                                          .join(jugadores_argentinos)\
                                          .map(lambda x: (x[1][0][0], (x[0], x[1][1], x[1][0][1])))

variacion_50 = estadisticas_argentinos.join(partidos)\
                                      .map(lambda x: ((x[1][0][0], x[1][0][1], (x[1][1][0], x[1][1][1])), (x[1][0][2], 1)))\
                                      .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]) )\
                                      .map(lambda x: ((x[0][0], x[0][1]), (x[0][2], x[1][0]/x[1][1])) )\
                                      .reduceByKey(lambda x,y : (x,y))\
                                      .filter(lambda x: filter_func(x[1]))

variacion_50.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 46.0 failed 1 times, most recent failure: Lost task 4.0 in stage 46.0 (TID 171, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iter